In [ ]:
from library_import import *
from config_path import PATH
warnings.simplefilter("ignore")


In [ ]:
def get_individual_source(source, rentree):
    import pandas as pd, zipfile
    from modules.sise_content import src_load

    filename = f'{source}{str(rentree)[2:4]}'
    print(filename)

    from config_path import PATH
    with zipfile.ZipFile(f"{PATH}input/parquet_origine.zip", 'r') as z:
        df = pd.read_parquet(z.open(f'{filename}.parquet'), engine='pyarrow')
    return df

In [ ]:
df=get_individual_source('mana', '2013')

In [ ]:
df.TYPREPA.unique()

In [ ]:
global CONF
CONF=json.load(open('utils/config_sise.json', 'r'))

In [ ]:
# ALL_RENTREES = list(range(2004, 2004+1))
ALL_RENTREES = [2013]
for rentree in ALL_RENTREES:
    df_all = pd.DataFrame()
    # sources = get_sources(rentree)
    sources = ['mana']
    for source in sources:

        filename = f'{source}{str(rentree)[2:4]}'
        print(filename)


        # chargement des tables en conservant que les variables de la liste utils/vars_list
        df = src_load(filename, source, rentree)

        # to check the data from the new datasets
        # with pd.ExcelWriter(excel_path, mode='a', if_sheet_exists="replace") as writer:  
        #     pd.DataFrame({"name": df.columns, "non-nulls": len(df)-df.isnull().sum().values, "nulls": df.isnull().sum().values}).to_excel(writer, sheet_name=filename, index=False)

        # df_all = pd.concat([df_all, df], ignore_index=True)
    # df_all = correctif_vars(df_all)


In [ ]:
df.typrepa.unique()

In [ ]:
# df_all.head(5)
df_all.value_counts(['rentree', 'source'])

In [ ]:
df_all.net.unique()

In [ ]:
# df.info()
df_all.head(5)
# df.voie.unique()

In [ ]:
[conf.get('var_sise') for conf in CONF]

In [ ]:
CORRECTIFS_dict = get_all_correctifs()
BCN = bcn_complete()

In [ ]:


def vars_sise_to_be_check(year, bcn, com):
    from config_path import PATH
    # from utils.vars_in_nomen import vbcn
    import pandas as pd, json

    CONF=json.load(open('utils/config_sise.json', 'r'))
    vars_sise = pd.read_pickle(f"{PATH}output/items_by_vars{year}.pkl", compression='gzip')

    hors_nomen=pd.DataFrame()

    for conf in CONF:
        var_sise = conf["var_sise"]
        nomen = conf["n_data"]

        if nomen:
            print(f"#### {var_sise}")
            if var_sise in ['cometa', 'comins']:
                l=pd.DataFrame.from_dict(com[nomen]).iloc[:,0].unique()
            else:
                if var_sise in bcn[nomen].columns:
                    l=bcn[nomen][var_sise].unique()
                else:
                    print(f"- le nom de variable {var_sise} n'existe pas dans {nomen}\n - le code suivant va extraire la 1ere colonne {bcn[nomen].columns[0]}")
                    l=bcn[nomen].iloc[:,0].unique()
                
            tmp=vars_sise.loc[(vars_sise.variable==var_sise)].assign(nomenclature=nomen)
            tmp.loc[~tmp.item.isin(l), 'hors_nomenclature'] = '1'
            hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)  

        # hors_nomen.to_csv(f"{PATH}test/vars_hs_nomen.csv", sep=';', encoding='utf-8', index=False, na_rep='', float_format='str')
   
    return hors_nomen
vars_sise_to_be_check(2024, BCN, CORRECTIFS_dict)


In [ ]:

hors_nomen=pd.DataFrame()
for k,v in vbcn.items():
    bcn[v].columns=bcn[v].columns.str.lower()
    if k in bcn[v].columns:
        l=bcn[v][k].unique()
        # tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))].value_counts(vs_cols_base, dropna=False).reset_index()
        # hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)
    else:
        print(f"- le nom de variable {k} n'existe pas dans {v}\nla 1ere colonne est {bcn[v].iloc[:,0]}, le code suivant va extraire la 1ere colonne")
        l=bcn[v].iloc[:,0].unique()
        tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))]
        hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=True)      

hors_nomen=hors_nomen.pivot_table(index=['rentree', 'variable', 'item'], columns='source',   values='count', 
    aggfunc='sum').reset_index()
hors_nomen.fillna('',inplace=True) 

In [ ]:

# work_csv(hors_nomen, 'modalites_hsn')

In [ ]:

test=pd.DataFrame()
for k,v in vbcn.items():
    bcn[v].columns=bcn[v].columns.str.lower()
    l=bcn[v].iloc[:,0].unique()
    tmp=vars_sise.loc[(vars_sise.variable==k)&(~vars_sise.item.isin(l))].value_counts(vs_cols_base, dropna=False).reset_index()
    # hors_nomen=pd.concat([hors_nomen, tmp], ignore_index=T:rue)   

In [ ]:
# for k,v in vbcn.items():
#     bcn[v].columns=bcn[v].columns.str.lower()
#     l=bcn[v].iloc[:,0].unique()
#     print(f"{v} -> {l}")

In [ ]:
for i in bcn:
    x=bcn[i]
    print(f"{i} -> {x.iloc[:,0].unique()}")


In [ ]:
bcn['N_SPECIALISATION'][bcn['N_SPECIALISATION'].specialisation=='0000']

In [ ]:
work_csv(bcn['N_PARCOURS_TYPE_SISE'], 'parcours')

In [ ]:
t=pd.read_csv(f"{PATH}sise_config.csv", encoding='utf-8', na_values=' ', keep_default_na=False, sep=';', dtype='str')
t.to_json('utils/config_sise.json', orient='records', compression='infer', indent=4)

In [ ]:
from utils.config import sise_config
CONF=sise_config()

In [ ]:
import json
json.load(open('utils/config_sise.json', 'r'))